# Amazon Mechanical Turk

The role of this script is to generate the data in a format that we can use to label on Mechanical Turk

1. Generate the next batch
2. Read the batch and write it to MongoDB

In [5]:
from data.dao import DataAccess, LabelGetter
from data.turk import TurkResults2Label

import pandas as pd
import datetime
import re

%matplotlib inline

Getting the data from DataAccess is so easy!

In [6]:
X = DataAccess.get_not_labeled().sort("random_number")
X.head(10)

,predict,random_number,text
_id,,,
557a1d0ad6dfbb4a90fab2d5,0.211107,0.004158,@tromepe http://t.co/96kFPCJnEp Acaso Rodrigo ...
5583bf23d6dfbb206a4387fa,0.823236,0.004167,I move in slowmo when in drunk
558dd892d6dfbb0a948198e6,0.819184,0.004169,drunk @ work lol
55737859d6dfbb021809f360,0.238229,0.004179,Our silly Saturday morning beginners are wishi...
55735667d6dfbb5f93005eb3,0.396975,0.004182,😂😂😂😂😂 no vale
55770442d6dfbb179bf05a10,0.220469,0.004182,We are looking for a buyer on 6845 Brigham Bay...
556fc13ad6dfbb11fa70a0a7,0.499303,0.004184,I need alcohol in me like soon! It's been a while
5574e6c7d6dfbb5ab8822d46,0.460629,0.004190,#confessionnight one time at 2 am i made waffl...
556e8debd6dfbb3061976b0e,0.215714,0.004196,My #TeenChoice nominee for #ChoiceMaleWebStar ...


# Generating...

In [7]:
date = str(datetime.date.today())
candidate = X.sort("random_number").head(500)
key = candidate.random_number.apply(hash).sum()

In [8]:
candidate["text"] = candidate.text.str.encode("utf-8")

In [9]:
candidate[["text", "random_number"]].to_csv(
    "../amt/amt|{}|{}.csv".format(date, key),
)

In [10]:
%%bash

ls ../amt/

Batch_2055156_batch_results.csv
Batch_2064123_batch_results.csv
amt_2015-08-13_4952308500878643456.csv
amt_2015-08-17_2546340314496401664.csv
amt_2015-08-20_8806137010034599936.csv
amt|2015-09-10|6072541214091395328.csv


# Parsing to Label

#### New Labels

Straight from Amazon Mechanical Turk

In [24]:
df = pd.concat([pd.DataFrame.from_csv("../amt/Batch_2064125_batch_results.csv"),
      pd.DataFrame.from_csv("../amt/Batch_2064123_batch_results.csv")]).set_index("_id")
df = df[df.Worker != "A3EBA6G9AG7CO9"]

df.Answer.value_counts() / df.Answer.value_counts().sum()

Not Alcohol Related                               0.390244
First Person - Alcohol::Looking to drink          0.125508
First Person - Alcohol::Casual Drinking           0.116362
Alcohol Related::Discussion                       0.114837
First Person - Alcohol::Reflecting on drinking    0.113821
First Person - Alcohol::Heavy Drinking            0.097561
Alcohol Related::Promotional Content              0.041667
dtype: float64

#### Writing Labels using TurkResults2Label

In [25]:
%%time
DataAccess.write_labels(df.Answer.apply(TurkResults2Label.parse_to_labels))

CPU times: user 1.71 s, sys: 478 ms, total: 2.19 s
Wall time: 1min 16s
